In [6]:
from dotenv import load_dotenv
import openai
import os

load_dotenv()


True

In [7]:
MODEL = "gpt-3.5-turbo"

OUTPUT_FOLDER = "../generated_GUIs/"

In [8]:
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
    model = MODEL,
    messages=[
    {"role": "user", "content": "create the HTML code for a Webpage, with a bit of styling, that represents following functionality: screen showing contacts page of a call app"}    
    ]
)

In [9]:
with open(OUTPUT_FOLDER + "contact_list.html", 'w') as f:
    f.write(completion.choices[0].message.content)

